In [4]:
x = [[1,2],[3,4],[5,6],[7,8]]
y = [[3],[7],[11],[15]]

In [5]:
import torch
from torch import nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x).float().to(device)
        self.y = torch.tensor(y).float().to(device)
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]
    def __len__(self):
        return len(self.x)

In [7]:
ds = MyDataset(x, y)
dl = DataLoader(ds, batch_size=2, shuffle=True)

In [9]:
model = nn.Sequential(
    nn.Linear(2,8),
    nn.ReLU(),
    nn.Linear(8, 1)
).to(device)

In [10]:
!pip install torch_summary
from torchsummary import summary

In [13]:
summary(model, (1, 2))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1, 8]              24
              ReLU-2                 [-1, 1, 8]               0
            Linear-3                 [-1, 1, 1]               9
Total params: 33
Trainable params: 33
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [14]:
# training the model
loss_fn = nn.MSELoss()
from torch.optim import SGD
opt = SGD(model.parameters(), lr = 0.001)
import time
loss_values = []
start = time.time()
for _ in range(50):
    for ix, iy in dl:
        opt.zero_grad()
        loss_value = loss_fn(model(ix),iy)
        loss_value.backward()
        opt.step()
        loss_values.append(loss_value.item())
end = time.time()
print(end - start)

0.08603715896606445


In [17]:
# creating validation set
val =  torch.tensor([[8,9], [10,11], [1.5, 2.5]])

In [18]:
# make predictions
model(val)

tensor([[16.6186],
        [20.3485],
        [ 4.4821]], grad_fn=<AddmmBackward0>)

#### saving a model

In [19]:
# tranfer model to when saving cpu so that it can be loaded anywhere
torch.save(model.to("cpu").state_dict(), "mymodel.pth")

#### loading

In [23]:
# when loading, initialize the model then load in the weights
model = nn.Sequential(
    nn.Linear(2, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
).to(device)

In [24]:
state_dict = torch.load("mymodel.pth")

In [25]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [26]:
model(val)

tensor([[16.6186],
        [20.3485],
        [ 4.4821]], grad_fn=<AddmmBackward0>)